In [1]:
## Import functions for Data Scraping, Treatment and Plotting

## System
import re
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## EDA
import numpy as np
import pandas as pd

## Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')


In [14]:
## FUNCIONS

## IMPORT DATA FROM CSV FILE
def ImportData(team_file, type):
    folder = team_file + '/'
    maps_file = team_file + '_'+type+'_MAPS.csv'
    matches_file = team_file + '_'+type+'_MATCHES.csv'
    
    maps = pd.read_csv(CSV_PATH + folder + maps_file)
    matches = pd.read_csv(CSV_PATH + folder + matches_file)

    return maps, matches

## Get Map data for all teams
def AllTeamsMaps(files, type, team_col, filter_value):
    concat = pd.concat([ImportData(team_file, type)[0] for team_file in files])
    filter = FilterCol(concat, team_col, filter_value)
    
    filter[team_col][(filter[team_col] == 'NOOR') | (filter[team_col] == 'NOORG2.0')] = 'paiN'
    filter[team_col][filter[team_col] == 'Mix'] = 'IMP'
    filter[team_col][(filter[team_col] == 'Ingaming') | (filter[team_col] == 'NMDM')] = 'Inga'
    filter[team_col][filter[team_col] == 'FF'] = 'VORA'
    filter[team_col][filter[team_col] == 'HAVA'] = 'HAVAN'
    
    if type == 'overview': ExtraInfoMaps(filter)
    return filter

## Get Match data for all teams
def AllTeamsMatches(files, type, team_col, filter_value):
    concat = pd.concat([ImportData(team_file, type)[1] for team_file in files])
    filter = FilterCol(concat, team_col, filter_value)
    
    filter[team_col][(filter[team_col] == 'NOOR') | (filter[team_col] == 'NOORG2.0')] = 'paiN'
    filter[team_col][filter[team_col] == 'Mix'] = 'IMP'
    filter[team_col][(filter[team_col] == 'Ingaming') | (filter[team_col] == 'NMDM')] = 'Inga'
    filter[team_col][filter[team_col] == 'FF'] = 'VORA'
    filter[team_col][filter[team_col] == 'HAVA'] = 'HAVAN'
    
    if type == 'overview': ExtraInfoMatches(filter)
    return filter


## ADD EXTRA INFO TO MATCHES DATAFRAME -- OVERVIEW
def ExtraInfoMatches(df):
    # KDA + FK/FD per map
    df['KPM'] = round(df['K'] / df['Num_maps'],2)
    df['DPM'] = round(df['D'] / df['Num_maps'],2)
    df['APM'] = round(df['A'] / df['Num_maps'],2)
    df['FKPM'] = round(df['FK'] / df['Num_maps'],2)
    df['FDPM'] = round(df['FD'] / df['Num_maps'],2)

## ADD EXTRA INFO TO MAPS DATAFRAME -- OVERVIEW
def ExtraInfoMaps(df):

    # Total Rounds
    df['total_rounds'] = df['rounds_won'] + df['rounds_lost']

    # Round Win Rate
    df['win_rate'] = round(df['rounds_won'] / df['total_rounds'] * 100, 2)

    # KDA + FK/FD per Round
    df['KPR'] = round(df['K'] / df['total_rounds'],2)
    df['DPR'] = round(df['D'] / df['total_rounds'],2)
    df['APR'] = round(df['A'] / df['total_rounds'],2)
    df['FKPR'] = round(df['FK'] / df['total_rounds'],2)
    df['FDPR'] = round(df['FD'] / df['total_rounds'],2)

    # Won/Lost Map
    df['RESULT'] = ['W' if x == True
                    else 'L'
                    for x in df['rounds_won'] > df['rounds_lost']]

## FILTER FUNCTIONS BY GIVEN OPPONENT/MAP/PLAYER/AGENT

def FilterCol(df, col, values):
    return df[df[col].isin(values)].reset_index(drop=True)


## CORRELATION OF GIVEN FEATURE WITH NUMBER OF ROUNDS WON
def WinCorr(df, stats, filter_maps = None):
    if filter_maps is not None:
        df = FilterMap(df, filter_maps)
    group = maps.groupby('rounds_won').mean()[stats].reset_index()
    corr_pearson = (group.corr())['rounds_won'][stats]
    corr_spearman = (group.corr(method='spearman'))['rounds_won'][stats]
    corr_kendall = (group.corr(method='kendall'))['rounds_won'][stats]
    print("Correlação Pearson média({0}) por Rounds Vencidos: {1}".format(stats, round(corr_pearson, 2)))
    print("Correlação Spearman média({0}) por Rounds Vencidos: {1}".format(stats, round(corr_spearman, 2)))
    print("Correlação Kendall média({0}) por Rounds Vencidos: {1}".format(stats, round(corr_kendall, 2)))

## CREATE DICT WITH MULTIPLE DATAFRAMES BY FILTERED VALUES
def FilterDict(col):
    aux = {}
    names = maps[col].unique()
    for name in names:
        aux[name] = maps[maps[col] == name].reset_index(drop=True)
    return aux

def Boxplot(nome_col, value):
    sns.set(rc={'figure.figsize':(12,7)})
    sns.boxplot(x=maps[nome_col], y=maps[value], order=maps[nome_col].sort_values().unique())

def Histogram(data ,value_col, filter):
    sns.set(rc={'figure.figsize':(12,7)})
    sns.displot(data=data, x=value_col, col=filter)

def PlotAgentPickRate(df):
    for i, map in enumerate(MAPS):
        filter = FilterMap(df, [map])
        agent_pick_rate = filter['Agents'].value_counts() / (len(filter) / 5)  * 100
        plt.figure(i, figsize=(10,))
        sns.barplot(y=agent_pick_rate.index, x=agent_pick_rate.values, palette="Blues_d").set_title('Agent Pick Rate -- '+map+' -- Played: '+str(int(len(filter) / 5))+' times')

In [2]:
ALL_FILES = ['gamelanders', 'pain', 'vorax', 'havan', 'imperial', 'ingaming', 'vikings', 'black_dragons']
ALL_TEAMS = ['GL', 'IMP','Mix', 'HAVAN', 'HAVA', 'Inga', 'Ingaming', 'NMDM', 'VKS', 'FF', 'VORA', 'BD', 'paiN','NOOR', 'NOORG2.0']
MAPS = ['ASCENT', 'BIND', 'HAVEN', 'ICEBOX', 'SPLIT']

CSV_PATH = '../data/csv/'
EXCEL_PATH = '..data/excel/'

In [53]:
overview_maps = AllTeamsMaps(ALL_FILES,'overview', 'Team', ALL_TEAMS)
performance_maps = AllTeamsMaps(ALL_FILES, 'performance', 'Team', ALL_TEAMS)
economy_maps = AllTeamsMaps(ALL_FILES, 'economy', 'Team', ALL_TEAMS)

In [19]:
overview_maps.head()

,Player,Agents,ACS,K,D,A,KD_DIFF,ADR,HS%,FK,...,Map,Patch,total_rounds,win_rate,KPR,DPR,APR,FKPR,FDPR,RESULT
0,Jonn,jett,398,22,9,5,13,224.6,18,3,...,SPLIT,1.12,16,81.25,1.38,0.56,0.31,0.19,0.06,W
1,mwzera,raze,268,14,12,5,2,181.0,17,2,...,SPLIT,1.12,16,81.25,0.88,0.75,0.31,0.12,0.06,W
2,JhoW,omen,185,13,8,7,5,103.9,19,1,...,SPLIT,1.12,16,81.25,0.81,0.50,0.44,0.06,0.06,W
3,fznnn,cypher,155,8,9,9,-1,83.6,14,1,...,SPLIT,1.12,16,81.25,0.50,0.56,0.56,0.06,0.31,W
4,Nyang,breach,131,8,10,3,-2,66.6,15,0,...,SPLIT,1.12,16,81.25,0.50,0.62,0.19,0.00,0.06,W


In [27]:
economy_maps.head()

,Team,Pistol_W,Pistol_P,Pistol_WR,Eco_0_5_P,Eco_0_5_W,Eco_0_5_WR,Semi_Eco_5_10_P,Semi_Eco_5_10_W,Semi_Eco_5_10_WR,...,Semi_Buy_10_20_W,Semi_Buy_10_20_WR,Full_Buy_20_P,Full_Buy_20_W,Full_Buy_20_WR,Opp_team,Num_maps,Patch,match_id,Map
0,GL,2,2,100.0,2,2,100.0,1,1,100.0,...,3,75.0,9,7,78.0,IMP,1.0,1.12,6797,SPLIT
1,IMP,0,2,0.0,3,0,0.0,2,0,0.0,...,2,33.0,5,1,20.0,GL,1.0,1.12,6797,SPLIT
2,GL,2,2,100.0,2,2,100.0,2,1,50.0,...,3,60.0,13,7,54.0,IMP,1.0,1.12,6797,BIND
3,IMP,0,2,0.0,4,0,0.0,2,0,0.0,...,2,50.0,12,7,58.0,GL,1.0,1.12,6797,BIND
4,GL,1,2,50.0,3,1,33.0,0,0,NaN,...,2,100.0,11,10,91.0,RED,1.0,NaN,6526,ASCENT


In [47]:
## Merge Overview + resto

merge_over_perf = overview_maps.merge(performance_maps, how='inner', left_on=['match_id','Map','Player'],right_on=['match_id','Map','Player']).drop_duplicates()
base_geral = merge_over_perf.merge(economy_maps, how='inner', left_on=['match_id','Map','Team_x'],right_on=['match_id','Map','Team']).drop_duplicates()

In [49]:
base_geral.to_csv('base_geral.csv', index=False)